In [20]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# NLTK Stop words
from nltk.corpus import stopwords
import nltk


import spacy
from FedTools import FederalReserveMins


In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dsimbandumwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Functions

In [23]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [24]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Load Dataset

# LoadDataset
df=pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

In [25]:
fed_mins = FederalReserveMins(
            main_url = 'https://www.federalreserve.gov', 
            calendar_url ='https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm',
            start_year = 2015,        
            historical_split = 2017,
            verbose = True,
            thread_num = 10)

df = fed_mins.find_minutes()

Constructing links between 2015 and 2023


/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/FedTools/FedMins.py:93: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  historical_mins = fed_mins_annual_soup.findAll('a', text='HTML')
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/FedTools/FedMins.py:94: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  historical_mins.extend(fed_mins_annual_soup.findAll('a', text='Minutes'))
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/FedTools/FedMins.py:93: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  historical_mins = fed_mins_annual_soup.findAll('a', text='HTML')
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/FedTools/FedMins.py:94: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' ins

Extracting Federal Reserve Minutes.
Retrieving articles.
.............

/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/FedTools/FedMins.py:93: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  historical_mins = fed_mins_annual_soup.findAll('a', text='HTML')
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/FedTools/FedMins.py:94: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  historical_mins.extend(fed_mins_annual_soup.findAll('a', text='Minutes'))


......................................................

In [26]:
df.rename(columns={'Federal_Reserve_Mins': 'content'}, inplace=True)
df

,content
2015-01-28,"The Federal Reserve, the central bank of the U..."
2015-03-18,"The Federal Reserve, the central bank of the U..."
2015-04-29,"The Federal Reserve, the central bank of the U..."
2015-06-17,"The Federal Reserve, the central bank of the U..."
2015-07-29,"The Federal Reserve, the central bank of the U..."
...,...
2022-11-02,"The Federal Reserve, the central bank of the U..."
2022-12-14,"The Federal Reserve, the central bank of the U..."
2023-02-01,"The Federal Reserve, the central bank of the U..."
2023-03-22,"The Federal Reserve, the central bank of the U..."


In [27]:
# Convert to list 
data = df.content.values.tolist()  
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
#pprint(data[:1])

<>:4: DeprecationWarning: invalid escape sequence '\S'
<>:6: DeprecationWarning: invalid escape sequence '\s'
<>:4: DeprecationWarning: invalid escape sequence '\S'
<>:6: DeprecationWarning: invalid escape sequence '\s'
/var/folders/vm/70vjm63s4j19_zfv4zc5lz480000gn/T/ipykernel_12576/3997535065.py:4: DeprecationWarning: invalid escape sequence '\S'
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
/var/folders/vm/70vjm63s4j19_zfv4zc5lz480000gn/T/ipykernel_12576/3997535065.py:6: DeprecationWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


# Clean Data

In [28]:
#deacc=True removes punctuations
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [29]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

In [30]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser','ner'])
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/dsimbandumwe/anaconda3/envs/TopicModel/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_EN

# tmp

# Create Dictionary and Corpus needed for Topic Modeling

In [31]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized) 

In [32]:
# Create Corpus 
texts = data_lemmatized  

In [33]:
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
#print(corpus[:1])

In [34]:
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Building topic model

In [35]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [36]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"rate" + 0.018*"inflation" + 0.016*"participant" + 0.015*"market" + '
  '0.011*"policy" + 0.010*"economic" + 0.010*"federal" + 0.009*"percent" + '
  '0.009*"committee" + 0.008*"continue"'),
 (1,
  '0.016*"committee" + 0.016*"market" + 0.013*"rate" + 0.013*"inflation" + '
  '0.012*"participant" + 0.012*"foreign" + 0.010*"policy" + 0.009*"economic" + '
  '0.008*"bank" + 0.007*"financial"'),
 (2,
  '0.018*"inflation" + 0.017*"rate" + 0.017*"market" + 0.013*"committee" + '
  '0.013*"economic" + 0.012*"policy" + 0.011*"price" + 0.011*"participant" + '
  '0.009*"increase" + 0.009*"continue"'),
 (3,
  '0.017*"participant" + 0.016*"inflation" + 0.015*"market" + 0.014*"rate" + '
  '0.011*"policy" + 0.010*"remain" + 0.009*"economic" + 0.009*"price" + '
  '0.009*"committee" + 0.009*"continue"')]


# Evaluate topic models

In [37]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.577516755251496

Coherence Score:  0.35157724198169105


In [38]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.011485 -0.036105       1        1  32.885056
2      0.018341  0.025452       2        1  26.607622
0      0.012792  0.007252       3        1  24.698540
1     -0.042618  0.003400       4        1  15.808782, topic_info=             Term         Freq        Total Category  logprob  loglift
472       foreign  1152.000000  1152.000000  Default  30.0000  30.0000
269      currency   527.000000   527.000000  Default  29.0000  29.0000
189     committee  3417.000000  3417.000000  Default  28.0000  28.0000
1164  transaction   526.000000   526.000000  Default  27.0000  27.0000
1008       select   256.000000   256.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
650          long   248.043025  1714.904962   Topic4  -5.2991  -0.0889
801       percent   268.189836  2384.210076   Topic4  -5.2210  -0.3403
559      increase   263.588803  2536.464684   Topic4  -5.2383  -0.4195
915        recent   235.044650  1514.074342   Topic4  -5.3529  -0.0182
487          fund   236.457169  1789.618956   Topic4  -5.3469  -0.1794

[359 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
3002      1  0.077020          acceptable
3002      2  0.038510          acceptable
3002      3  0.731688          acceptable
3002      4  0.154039          acceptable
13        4  0.917459  accounts_maintaine
...     ...       ...                 ...
4073      4  0.075512       weigh_heavily
1239      1  0.297044                year
1239      2  0.294798                year
1239      3  0.259983                year
1239      4  0.148241                year

[779 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])